# ADA Project Milestone 2 
## Project proposal and initial analyses

### Import packages

In [38]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set()
sns.set_style("whitegrid")
colors = sns.color_palette("colorblind", 16)

### Import Datasets

In [9]:
DATA_FOLDER = '../data/'

BEER_ADVOCATE = DATA_FOLDER + "BeerAdvocate/"
MATCHED_BEER_DATA = DATA_FOLDER + "MatchedBeerData/"
RATE_BEER = DATA_FOLDER + "RateBeer/"

## Beer Advocate Data

In [22]:
# Read Beer Advocate folder's datasets

beers_ba = pd.read_csv(BEER_ADVOCATE + "beers.csv")
breweries_ba = pd.read_csv(BEER_ADVOCATE + "breweries.csv")
users_ba = pd.read_csv(BEER_ADVOCATE + "users.csv")

In [27]:
beers_ba.sample(5)

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
68760,43185,Spike's 2008,16386,O'so Brewing Company & Tap House,American Amber / Red Ale,3,2,4.06,NaN,NaN,NaN,4.01,NaN,0,NaN
143544,67047,Brewer's Alley Chocolate Ale,1000,Brewer's Alley Restaurant & Brewery,American Brown Ale,1,1,4.31,NaN,NaN,NaN,4.31,NaN,0,NaN
69963,264372,Bourbon Barrel Aged Naughty Nick,1550,Rock Bottom Restaurant & Brewery,Scottish Ale,1,1,3.79,NaN,NaN,9.0,3.79,NaN,0,NaN
246603,209837,Root Bier,43739,Intrinsic Brewing,Herbed / Spiced Beer,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
82873,278952,Mister Tea,33018,The Rare Barrel,American Wild Ale,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [28]:
users_ba.head()

,nbr_ratings,nbr_reviews,user_id,user_name,joined,location
0,7820,465,nmann08.184925,nmann08,1.199704e+09,"United States, Washington"
1,2521,2504,stjamesgate.163714,StJamesGate,1.191838e+09,"United States, New York"
2,1797,1143,mdagnew.19527,mdagnew,1.116410e+09,Northern Ireland
3,31,31,helloloser12345.10867,helloloser12345,1.101380e+09,Northern Ireland
4,604,604,cypressbob.3708,cypressbob,1.069326e+09,Northern Ireland


In [34]:
breweries_ba.sort_values("nbr_beers",ascending=False)

,id,location,name,nbr_beers
10691,2837,"United States, Pennsylvania",Rock Bottom Restaurant and Brewery,1196
14031,16083,"United States, Illinois",Rock Bottom Restaurant and Brewery,1196
15759,1099,"United States, Arizona",Rock Bottom Restaurant & Brewery,1196
10542,1100,"United States, Arizona",Rock Bottom Restaurant & Brewery - Arrowhead,1196
8106,2981,"United States, Wisconsin",Rock Bottom Restaurant & Brewery,1196
...,...,...,...,...
4139,24527,Germany,Karls Brauhaus,0
13730,48529,"United States, Texas",Fredonia Brewery,0
4137,5785,Germany,Karl Lechler Brauerei,0
4136,7341,Germany,Karl Bindewald Kupfermühle GmbH,0


In [39]:
!head -n 20 C:\Users\User\Documents\EPFL Master semestre 3\Applied Data Analysis\ada-2022-project-toohottohoot\data\BeerAdvocate\reviews.txt

'head' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


## Matched Beer Data

In [42]:
# Read Matched Beer folder's datasets

beers_mb = pd.read_csv(MATCHED_BEER_DATA + "beers.csv")
breweries_mb = pd.read_csv(MATCHED_BEER_DATA + "breweries.csv")
users_mb = pd.read_csv(MATCHED_BEER_DATA + "users.csv")
ratings_mb = pd.read_csv(MATCHED_BEER_DATA + "ratings.csv")
users_approx_mb = pd.read_csv(MATCHED_BEER_DATA + "users_approx.csv")

In [43]:
ratings_mb.head()

,ba,ba.1,ba.2,ba.3,ba.4,ba.5,ba.6,ba.7,ba.8,ba.9,...,rb.6,rb.7,rb.8,rb.9,rb.10,rb.11,rb.12,rb.13,rb.14,rb.15
0,abv,appearance,aroma,beer_id,beer_name,brewery_id,brewery_name,date,overall,palate,...,brewery_name,date,overall,palate,rating,style,taste,text,user_id,user_name
1,11.3,4.5,4.5,645,Trappistes Rochefort 10,207,Brasserie de Rochefort,1324810800,5.0,4.5,...,Brasserie Rochefort,1387710000,19.0,4.0,4.6,Abt/Quadrupel,9.0,a) Geruch malzig-schwer-sÃ¼Ã. Riecht schon ...,83106,Erzengel
2,5.0,NaN,NaN,28191,Myanmar Lager Beer,9369,Myanmar Brewery and Distillery,1322650800,NaN,NaN,...,Myanmar Brewery and Distillery,1322564400,6.0,2.0,1.7,Pale Lager,4.0,"Can. Weak and watery, not the best beer of the...",91324,visionthing
3,5.0,3.5,3.5,57911,Cantillon Tyrnilambic Baie D’Argousier Lambic,388,Brasserie Cantillon,1344074400,4.0,4.0,...,Cantillon,1353582000,17.0,4.0,4.1,Lambic Style - Fruit,8.0,"Bottle @ One Pint Pub, Helsinki. Originally ra...",98624,tiong
4,5.0,4.0,3.5,57913,Cantillon Pikkulinnun Viskilambic,388,Brasserie Cantillon,1344074400,4.0,4.0,...,Cantillon,1416222000,16.0,4.0,4.1,Lambic Style - Unblended,9.0,"Draught @Â Pikkulintu, Helsinki, Finland. A pr...",98624,tiong


## Rate Beer Data

In [44]:
# Read Rate Beer folder's datasets

beers_rb = pd.read_csv(RATE_BEER + "beers.csv")
breweries_rb = pd.read_csv(RATE_BEER + "breweries.csv")
users_rb = pd.read_csv(RATE_BEER + "users.csv")